In [1]:
using CSV
using DataFrames
using Z3

In [29]:
"""
nslot = 17 # 時間帯割り当て
narea = 2 # エリア数
requirenum = 1 # 必要人数
totaltime = 15 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
"""

function maketable(csvfile ;nslot = 17, narea = 2, requirenum = 1, totaltime = 15, totalswitch = 2)
    ctx = Z3.Context()

    cons = []
    area = Dict()

    df = CSV.read(csvfile, DataFrame)

    nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end"] # 名前取得
    println(nms)

    for (i,nm) = nms
        tmparea = []
        for t = 1:nslot
            x = Z3.int_const(ctx, "var$(i)_$(t)")
            f = [Z3.int_const(ctx, "area$(k)_$(i-2)_$(t)") for k = 1:narea]
            push!(tmparea, f)
            for v = f
                push!(cons, v >= 0) # 割り当て変数非負条件
            end
            push!(cons, sum(f) <= x) # 可能時間帯に高々一つの割り当て
            if ismissing(df[t,i])
                push!(cons, x == 1) # 無記入は「可能」
            else
                push!(cons, x == Int(df[t,i])) # 可能な時間帯の指定
            end
        end
        area[nm] = tmparea
    end

    for t = 1:nslot
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for (_,xs) = area
                tmp += xs[t][k]
            end
            push!(cons, tmp == requirenum) # 各時間帯で必要人数確保
        end
    end

    for (nm,xs) = area
        tmp = Z3.int_val(ctx, 0)
        for x = xs
            tmp += sum(x)
        end
        push!(cons, tmp <= totaltime) # 労働時間制限
    end

    for (nm,xs) = area
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for t = 1:nslot
                if t == 1
                    tmp += xs[t][k]
                else
                    tmp += abs(xs[t][k] - xs[t-1][k])
                end
            end
            push!(cons, tmp <= totalswitch) # 場所替え数制約
        end
    end

    solver = Z3.Solver(ctx)
    for x = cons
        add(solver, x)
    end

    res = Z3.check(solver)
    if res == Z3.sat
        m = Z3.get_model(solver)
        result = Dict(["$k"=>Z3.get_numeral_int(v) for (k,v) = Z3.consts(m)])
    else
        error("No solution")
    end

    table = Dict()
    for (i,nm) = enumerate(keys(area))
        tmp = []
        for t = 1:nslot
            u = [result["area$(k)_$(i)_$(t)"] for k = 1:narea]
            res = findfirst(x->x==1, u)
            if isnothing(res)
                push!(tmp, 0)
            else
                push!(tmp, res)
            end
        end
        table[nm] = tmp
    end
    DataFrame(table)
end

maketable (generic function with 1 method)

In [30]:
maketable("testdata.csv", nslot = 17, narea = 2, requirenum = 1, totaltime = 15, totalswitch = 2)

[(3, "山田"), (4, "田中"), (5, "鈴木"), (6, "佐藤")]


Row,佐藤,山田,田中,鈴木
,Any,Any,Any,Any
1,0,0,2,1
2,2,1,0,0
3,2,1,0,0
4,1,2,0,0
5,0,2,1,0
6,0,0,1,2
7,0,0,1,2
8,0,0,1,2
9,0,0,1,2
